# Using a Pool to compute a matrix-matrix multiply

<hr/>

Your job in this problem is to improve the speed of the matrix-matrix multiply, implemented here using a Pool.  
You are free to be creative in writing your matrix-matrix multiply, but must follow a few ground rules. 

* You must use one of the Pool functions (map, map_async, apply or apply_async). 

* You cannot use any global variables

* You are free to re-write the way in which the matrix is distributed to processors.  For example, you can try tiling individual dot products to get better cache performance. 

* Your answer should agree with the `numpy.matmul` result to within about $10^{-10}$.  The code below computes the difference for you. 

To show the speed-up you get, write a second function (using `def`) that computes your faster matrix-matrix multiply and then report both the original results and your faster results.  

We are not looking for the "fastest" method to compute a matrix-matrix product (Numpy's `matmul` beats everything).  We are alos not looking for parallel speed-up.   Rather, this problem should give you a better feel as to how the multiprocessing Pool module works, and how to use it optimally.

If you are successful, you should see about a factor 5-10 (or more) speedup.  Try out your matrix-matrix multiply on matrices of dimensions $N = 2^7, 2^8, 2^9$ and $2^{10}$. 

To see how the Pool module works, see notebook posted on the course Wiki (Tuesday, Week 04). 

In [1]:
from multiprocessing import Pool
import numpy as np
import numpy.linalg as la
import itertools

In [2]:
def dot_ij(d):
    i,j,x,y = d
    return (i,j,np.dot(x,y))

def matmul_pool(A,B):
    m1,n1 = A.shape
    m2,n2 = B.shape
    if n1 != m2:
        print("Inner matrix dimensions do not agree")
        return None
    
    Bt = np.transpose(B)
    
    map_list = [(i,j,A[i],Bt[j]) for (i,j) in itertools.product(range(0,m1),range(0,n2))]    
    
    pool = Pool()
    results = []
    for d in map_list:
        r = pool.apply_async(func=dot_ij,args=(d,)) # updated with Pool.apply
        results.append(r)
    
    mulResult = [r.get() for r in results]
    C = np.empty((m1,n2),dtype='d')
    
    
    for r in mulResult:
        i,j,d = r
        C[i][j] = d
            
    return C

In [3]:
N = 2**8
np.random.seed(1234)
A = np.random.rand(N,N)
B = np.random.rand(N,N)

In [4]:
n_loop = 1
r_rep = 1
tr = %timeit -n $n_loop -r $r_rep -o pass; C_pool = matmul_pool(A,B)
time_loop = tr.best

# Get answer that we can compare to the numpy result
C_pool = matmul_pool(A,B)

print("")
print("{:>25s} {:12d}".format("Matrix dimensions (N)",N))
print("{:>25s} {:12.4e}".format("norm(C_pool-C_true)",la.norm(C_pool-np.matmul(A,B))))
print("{:>25s} {:12.6f}".format("Time using Pool (s)", time_loop))

1 loop, best of 1: 10.9 s per loop

    Matrix dimensions (N)          256
      norm(C_pool-C_true)   6.9284e-12
      Time using Pool (s)    10.934012


Result from pool.map for N = 8

1 loop, best of 1: 25.1 s per loop

    Matrix dimensions (N)          256
      norm(C_pool-C_true)   6.9284e-12
      Time using Pool (s)    25.147193

## Result:
    
Pool.apply was updated to pool.apply_async. Speed up in the range of from 2.5 to 4 times was achieved. 